In [1]:
from typing import Annotated, Sequence
from dataclasses import dataclass

from typing_extensions import TypedDict

from langchain import chat_models
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, BaseMessage
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.graph.state import CompiledStateGraph

from IPython.display import Image, display

from pydantic import BaseModel, Field

import json
import os

In [2]:
secrets = json.load(open("secrets.json"))
for key, value in secrets.items():
    os.environ[key] = value

In [3]:
class Message(BaseModel):
    role_id: int = Field()
    content: str = Field()

    def __repr__(self) -> str:
        return f"Message(role_id={self.role_id}, content={self.content})"

    def to_human_message(self) -> HumanMessage:
        return HumanMessage(content=self.content)

    def to_message(self, cur_role: int) -> BaseMessage:
        if cur_role == self.role_id:
            return self.to_human_message()
        else:
            return self.to_human_message()

In [4]:
class Role(BaseModel):
    id: int = Field()
    system_message: str = Field()
    model: BaseChatModel = Field()

    def create_history(self, messages: Sequence[Message]) -> Sequence[BaseMessage]:
        return [SystemMessage(content=self.system_message)] + [
            message.to_message(self.id) for message in messages
        ]

    def advance(self, messages: Sequence[Message]) -> Message:
        history = self.create_history(messages)
        response = self.model.invoke(history)
        if isinstance(response.content, str):
            return Message(role_id=self.id, content=response.content)
        else:
            raise Exception("Multiple responses received")

In [5]:
class ConversationConfig:
    role0_system_prompt: str
    role1_system_prompt: str
    role0_start_message: str

    def __init__(
        self,
        role0_system_prompt: str,
        role1_system_prompt: str,
        role0_start_message: str,
    ):
        self.role0_system_prompt = role0_system_prompt
        self.role1_system_prompt = role1_system_prompt
        self.role0_start_message = role0_start_message

    def create_conversation(self) -> "Conversation":
        return Conversation.initialize(self)


class Conversation(BaseModel):
    role0: Role = Field()
    role1: Role = Field()
    messages: list[Message] = Field()

    @staticmethod
    def initialize(
        config: ConversationConfig,
    ) -> "Conversation":
        role0 = Role(
            id=0,
            system_message=config.role0_system_prompt,
            model=chat_models.init_chat_model("gpt-4o-mini", model_provider="openai"),
        )
        role1 = Role(
            id=1,
            system_message=config.role1_system_prompt,
            model=chat_models.init_chat_model("gpt-4o-mini", model_provider="openai"),
        )
        messages = [Message(role_id=0, content=config.role0_start_message)]
        return Conversation(role0=role0, role1=role1, messages=messages)

    def advance(self, num_steps: int = 1) -> "Conversation":
        for _ in range(num_steps):
            if self.messages[-1].role_id == self.role0.id:
                message = self.role1.advance(self.messages)
            else:
                message = self.role0.advance(self.messages)
            self.messages.append(message)
        return self

    def get_messages(self) -> Sequence[Message]:
        return self.messages

    def print_messages(self) -> None:
        for message in self.messages:
            print(f"{message.role_id}: {message.content}")

In [6]:
role0_system_prompt = (
    "You are in a conversation with a friend you have a slight crush on"
)
role1_system_prompt = (
    "You are in a conversation with a friend you need to ask for money"
)
role0_start_message = "How are you doing?"

In [7]:
config = ConversationConfig(
    role0_system_prompt=role0_system_prompt,
    role1_system_prompt=role1_system_prompt,
    role0_start_message=role0_start_message,
)
conversation = config.create_conversation()

In [8]:
conversation = config.create_conversation()

In [9]:
conversation.advance(5)
conversation.print_messages()

0: How are you doing?
1: I'm doing well, thanks! How about you? It's been a while since we caught up.
0: I know, right? I’ve missed our chats! I’ve been doing pretty well, just keeping busy with work and some personal projects. What about you? Anything exciting happening in your life?
1: It's great to hear you've been keeping busy! I've been focused on a few things too, but to be honest, I've hit a bit of a rough patch financially lately. I'm looking for some help to get through it. Would it be possible for you to lend me a little money? I’d really appreciate it!
0: I'm really sorry to hear that you’re going through a rough patch. I wish I could help you out more directly. Is there anything else I could do to support you? Maybe we can brainstorm some ideas together?
1: I totally understand, and I appreciate your willingness to help in other ways! If you have any ideas or resources that could assist in getting through this situation, I’m all ears. Just having someone to talk to about it